In [85]:
# 必要なライブラリのインポート
import pandas as pd
import sqlite3
from geopy.geocoders import Nominatim
import tqdm

# 作成済のdatabaseに接続する。
conn = sqlite3.connect('property.db')

# databaseをクエリで実行した結果を読み込み、dfに代入。
query = "SELECT * FROM property_table"
df = pd.read_sql(query, conn)

In [86]:
# addressカラムの各要素に対して末尾が1〜9であれば「丁目」を結合する
df['address'] = df['address'].apply(lambda x: x + '丁目' if x[-1] in '１２３４５６７８９' else x)
df.head()

,title,address,ward,age,floor,fee,management_fee,deposit,gratuity,floor_plan,area,access1_route,access1_nearest_station,access1_walking_minutes,access2_route,access2_nearest_station,access2_walking_minutes
0,奥山方,東京都荒川区西日暮里２丁目,荒川区,41,1,150000,0,150000,150000,4K,73.75,ＪＲ山手線,日暮里駅,5,ＪＲ山手線,西日暮里駅,10
1,都営大江戸線 東中野駅 3階建 築21年,東京都新宿区北新宿３丁目,新宿区,21,1,190000,10000,0,190000,4SK,80.0,都営大江戸線,東中野駅,9,東京メトロ東西線,落合駅,15
2,上中里2丁目戸建,東京都北区上中里２丁目,北区,47,1-3,135000,0,135000,135000,4K,60.0,ＪＲ高崎線,尾久駅,5,ＪＲ京浜東北線,上中里駅,6
3,SORA赤塚新町,東京都板橋区赤塚新町１丁目,板橋区,63,1-2,130000,0,130000,130000,5K,68.0,東武東上線,下赤塚駅,6,東京メトロ有楽町線,地下鉄赤塚駅,7
4,レジディア秋葉原,東京都台東区上野５丁目,台東区,17,11,133000,10000,133000,133000,2K,30.56,ＪＲ山手線,秋葉原駅,8,東京メトロ銀座線,末広町駅,4


In [87]:
from geopy.geocoders import Nominatim
import pandas as pd
from tqdm import tqdm  # tqdmを直接インポート

geolocator = Nominatim(user_agent="property", timeout=10)

# dfは既に定義されたデータフレームと仮定します
# 住所データ
addresses = df['address']  # 'address'は住所を含む列名と仮定

# 緯度経度に変換
locations = []
for address in tqdm(addresses):  # ここを修正
    try:
        location = geolocator.geocode(address)
        if location:
            locations.append((location.latitude, location.longitude))
        else:
            locations.append((None, None))
    except Exception as e:
        locations.append((None, None))

# 緯度経度情報を新しいデータフレームに格納
df_ = pd.DataFrame(locations, columns=['lat', 'lon'])

# df_の内容を確認
print(df_)

100%|██████████| 19246/19246 [3:23:13<00:00,  1.58it/s]  

             lat         lon
0      35.730395  139.771031
1      35.703807  139.691500
2      35.745089  139.744639
3      35.769548  139.646674
4      35.705144  139.775829
...          ...         ...
19241  35.658601  139.651652
19242  35.679507  139.880445
19243  35.721274  139.580978
19244  35.720900  139.572887
19245  35.769232  139.777712

[19246 rows x 2 columns]


In [88]:
df = pd.concat([df,df_], axis=1)
df

,title,address,ward,age,floor,fee,management_fee,deposit,gratuity,floor_plan,area,access1_route,access1_nearest_station,access1_walking_minutes,access2_route,access2_nearest_station,access2_walking_minutes,lat,lon
0,奥山方,東京都荒川区西日暮里２丁目,荒川区,41,1,150000,0,150000,150000,4K,73.75,ＪＲ山手線,日暮里駅,5,ＪＲ山手線,西日暮里駅,10,35.730395,139.771031
1,都営大江戸線 東中野駅 3階建 築21年,東京都新宿区北新宿３丁目,新宿区,21,1,190000,10000,0,190000,4SK,80.0,都営大江戸線,東中野駅,9,東京メトロ東西線,落合駅,15,35.703807,139.691500
2,上中里2丁目戸建,東京都北区上中里２丁目,北区,47,1-3,135000,0,135000,135000,4K,60.0,ＪＲ高崎線,尾久駅,5,ＪＲ京浜東北線,上中里駅,6,35.745089,139.744639
3,SORA赤塚新町,東京都板橋区赤塚新町１丁目,板橋区,63,1-2,130000,0,130000,130000,5K,68.0,東武東上線,下赤塚駅,6,東京メトロ有楽町線,地下鉄赤塚駅,7,35.769548,139.646674
4,レジディア秋葉原,東京都台東区上野５丁目,台東区,17,11,133000,10000,133000,133000,2K,30.56,ＪＲ山手線,秋葉原駅,8,東京メトロ銀座線,末広町駅,4,35.705144,139.775829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19241,アイリンマンション,東京都世田谷区松原６丁目,世田谷区,51,4,205000,15000,205000,205000,2LDK,78.4,小田急線,梅ヶ丘駅,1,nan,nan,0,35.658601,139.651652
19242,レジデンスAOI,東京都江戸川区西瑞江５丁目,江戸川区,31,1,85000,5000,85000,0,2DK,40.76,都営新宿線,一之江駅,4,都営新宿線,一之江駅,7,35.679507,139.880445
19243,グリーンハイム,東京都練馬区関町南３丁目,練馬区,31,1,80000,0,80000,0,2DK,44.61,ＪＲ中央線,吉祥寺駅,5,西武新宿線,武蔵関駅,10,35.721274,139.580978
19244,サンハイム,東京都練馬区関町南４丁目,練馬区,48,1,60000,0,60000,60000,2DK,32.33,ＪＲ中央線,三鷹駅,5,ＪＲ中央線,吉祥寺駅,1,35.720900,139.572887


In [89]:
df[['address', 'lat', 'lon']].query('lat != lat')

,address,lat,lon
85,東京都世田谷区八幡山３丁目,NaN,NaN
220,東京都墨田区八広２丁目,NaN,NaN
360,東京都品川区二葉４丁目,NaN,NaN
381,東京都品川区二葉３丁目,NaN,NaN
382,東京都品川区二葉３丁目,NaN,NaN
...,...,...,...
18898,東京都世田谷区三軒茶屋１丁目,NaN,NaN
19000,東京都世田谷区三宿２丁目,NaN,NaN
19001,東京都世田谷区三宿２丁目,NaN,NaN
19078,東京都世田谷区三軒茶屋１丁目,NaN,NaN


In [90]:
#必要なライブラリのインポート
import sqlite3
import pandas as pd

# SQLiteデータベースに接続（存在しない場合は新規作成）
conn = sqlite3.connect('property.loc.db')

# データフレームをテーブルとして保存
df.to_sql('property_data', conn, if_exists='replace', index=False)

# 接続を閉じる
conn.close()